# **IBM APPLIED DATA SCIENCE CAPSTONE**

## WEEK 3: PEER GRADED ASSIGNMENT

**Import Libraries**

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

**1.Retrieve Data from Wikipedia**

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

**2.Data Cleaning Steps**

**- Convert data to Pandas Dataframe**

In [3]:
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood2'])

**-Clean the "Borough" Column**

In [4]:
# Find indexes of rows that have "Not assigned" in Borough column
indexNames = df[(df['Borough'] == "Not assigned")].index

# Drop rows that have "Not assigned" in Borough column
df.drop(indexNames,inplace=True)

# Drop the first row
df.dropna(inplace=True)

**-Collapse Data**

In [5]:
# Combine multiple rows into one row based on PostalCode and Borough
df=df.groupby(['PostalCode','Borough'])['Neighborhood2'].apply(', '.join).reset_index()

**-Clean Neighborhood Column**

In [6]:
# Replace "Not assigned" in Neighborhood column with the value in Borough column
def custom_fx(data):
    if data['Neighborhood2']=='Not assigned':
        var=data['Borough']
    else:
        var=data['Neighborhood2']
    return var

# Apply the function
df['Neighborhood']=df.apply(custom_fx,axis='columns')

# Check that there is no more "Not assigned" in Neighborhood column
print("There are {} rows that have 'Not assigned' in Neighborhood column in the dataframe".format(
    len(df[df['Neighborhood']=='Not assigned'])
)
     )

# Delete Neighborhood2 column
df.drop(columns='Neighborhood2')

There are 0 rows that have 'Not assigned' in Neighborhood column in the dataframe


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
# Export the dataframe
df.to_csv(r'D:\Learning\Applied Data Science\Course 4 Applied Data Science Capstone\Week 3\TRT.csv',index=None,header=True)

**3.Explore the Dataset**

In [8]:
# Shape of the table
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (df.shape,df.shape[0]))

The shape of the dataframe is (103, 4). The dataset has 103 rows.


## **TASK 2**

**Import Libraries**

In [9]:
import pandas as pd

**1.Read the Coordinates Table for Toronto**

In [15]:
# Geospatial_Coordinates provided in the instruction
lonlat=pd.read_csv('http://cocl.us/Geospatial_data')

# Toronto table created in task 1
trt=pd.read_csv('D:\Learning\Applied Data Science\Course 4 Applied Data Science Capstone\Week 3\TRT.csv')

**2.Merge the Two tables**

In [16]:
# Column names of lonlat
print("Column names of lonlat dataframe are: {}, {}, and {}.".format(lonlat.columns[0],lonlat.columns[1],lonlat.columns[2]))
print("Column names of trt dataframe are: {}, {}, and {}.".format(trt.columns[0],trt.columns[1],trt.columns[2]))

# Change the name "Postal Code" in lonlat to "PostalCode"
lonlat.rename(columns={'Postal Code':'PostalCode'},inplace=True)

# Left join
trt_geo=pd.merge(trt,lonlat,how='left',on='PostalCode')

Column names of lonlat dataframe are: Postal Code, Latitude, and Longitude.
Column names of trt dataframe are: PostalCode, Borough, and Neighborhood2.


**3.Export the Table**

In [28]:
# Export trt_geo for the preparation of Task 3
trt_geo.to_csv(r'D:\Learning\Applied Data Science\Course 4 Applied Data Science Capstone\Week 3\TRT_geo.csv',index=None,header=True)

**4.Explore the Table**

In [29]:
trt_geo

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [30]:
# Shape of the table
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (trt_geo.shape,trt_geo.shape[0]))

The shape of the dataframe is (103, 5). The dataset has 103 rows.
